In [7]:
from pathlib import Path
import pandas as pd
import numpy as np

import torch
import torchaudio
from speechbrain.pretrained import EncoderClassifier
from tqdm.auto import tqdm

In [4]:
DATA_PATH = Path("../data")
CLASSES = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
WEIGHTS_PATH = "TalTechNLP/voxlingua107-epaca-tdnn" # "speechbrain/google_speech_command_xvector"

In [16]:
classifier = EncoderClassifier.from_hparams(
    source=WEIGHTS_PATH, 
    savedir="pretrained_models/voxlingua107-epaca-tdnn",
    run_opts={"device":"cuda"},
)
audio_normalizer = classifier.audio_normalizer
label_encoder = classifier.hparams.label_encoder

In [6]:
out_prob, score, index, text_lab = classifier.classify_file(str(DATA_PATH / "test" / '0a0b032a8850131875bf.wav'))
print(text_lab)

['sa']


In [10]:
# def load_audio(path):
#     signal, sr = torchaudio.load(str(path), channels_first=False)
#     return audio_normalizer(signal, sr)

In [14]:
# audio_filepaths = sorted(list((DATA_PATH / "test").glob("*.wav")))

# wavs = [
#     load_audio(audiofile).unsqueeze(0)
#     for audiofile in tqdm(audio_filepaths)
# ]

  0%|          | 0/29620 [00:00<?, ?it/s]

In [19]:
# pred = []
# embeddings = []
# for wav in tqdm(wavs):
#     rel_length = torch.tensor([1.0])
#     embedding = classifier.encode_batch(wav, rel_length)
#     out_prob = classifier.mods.classifier(embedding).squeeze(1)
#     score, index = torch.max(out_prob, dim=-1)
#     text_lab = label_encoder.decode_torch(index)
#     pred.append(text_lab[0])
#     embeddings.append(embedding)

  0%|          | 0/29620 [00:00<?, ?it/s]

In [5]:
pred = []
audio_filepaths = sorted(list((DATA_PATH / "test").glob("*.wav")))
for audiofile in tqdm(audio_filepaths):
    out_prob, score, index, text_lab = classifier.classify_file(str(audiofile))
    pred.append(text_lab[0])

  0%|          | 0/29620 [00:00<?, ?it/s]

In [13]:
pd.Series(pred).value_counts()

yes        2904
down       2892
no         2868
right      2834
off        2796
up         2779
stop       2774
left       2761
on         2696
go         2670
silence    1351
unknown     295
dtype: int64

In [14]:
df = pd.DataFrame(data={
    "id": [a.stem for a in audio_filepaths],
    "category": pred,
})

In [15]:
df.to_csv("submission.csv", index=False)

In [ ]:
! rm *.wav